<a href="https://colab.research.google.com/github/prerakmathur05/RecommendationSystem/blob/main/Content_based_filtering_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from google.colab import files
import io

In [4]:
file1=files.upload()
file2=files.upload()

Saving movies.csv to movies.csv


Saving ratings.csv to ratings.csv


In [7]:
df_movies= pd.read_csv(io.BytesIO(file1['movies.csv']))
df_ratings= pd.read_csv(io.BytesIO(file2['ratings.csv']))

In [8]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
df_movies.drop(columns=['genres'], inplace=True)
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [14]:
df=pd.merge(df_movies,df_ratings, on = "movieId")
df.head()

,movieId,title,userId,rating,timestamp
0,1,Toy Story (1995),1,4.0,964982703
1,1,Toy Story (1995),5,4.0,847434962
2,1,Toy Story (1995),7,4.5,1106635946
3,1,Toy Story (1995),15,2.5,1510577970
4,1,Toy Story (1995),17,4.5,1305696483


In [15]:
df.shape

(100836, 5)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100836 non-null  int64  
 1   title      100836 non-null  object 
 2   userId     100836 non-null  int64  
 3   rating     100836 non-null  float64
 4   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 4.6+ MB


In [86]:
df.head()
g=df.groupby("title")
g.get_group("Forrest Gump (1994)")['rating'].sum()

1370.0

In [87]:
new_df = df.groupby('title')['rating'].sum().reset_index().rename(columns={'rating':"totalratingcount"})
new_df.sort_values(by='totalratingcount',ascending=False)

,title,totalratingcount
7593,"Shawshank Redemption, The (1994)",1404.0
3158,Forrest Gump (1994),1370.0
6865,Pulp Fiction (1994),1288.5
5512,"Matrix, The (1999)",1165.5
7680,"Silence of the Lambs, The (1991)",1161.0
...,...,...
9042,Uncle Nino (2003),0.5
5007,Legionnaire (1998),0.5
7145,Ring of Terror (1962),0.5
3483,Glitter (2001),0.5


In [88]:
final_df=pd.merge(df,new_df, on = 'title')
final_df.head()

,movieId,title,userId,rating,timestamp,totalratingcount
0,1,Toy Story (1995),1,4.0,964982703,843.0
1,1,Toy Story (1995),5,4.0,847434962,843.0
2,1,Toy Story (1995),7,4.5,1106635946,843.0
3,1,Toy Story (1995),15,2.5,1510577970,843.0
4,1,Toy Story (1995),17,4.5,1305696483,843.0


In [89]:
final_df.describe()

,movieId,userId,rating,timestamp,totalratingcount
count,100836.000000,100836.000000,100836.000000,1.008360e+05,100836.000000
mean,19435.295718,326.127564,3.501557,1.205946e+09,219.795336
std,35530.987199,182.618491,1.042529,2.162610e+08,251.531051
min,1.000000,1.000000,0.500000,8.281246e+08,0.500000
25%,1199.000000,177.000000,3.000000,1.019124e+09,43.000000
50%,2991.000000,325.000000,3.500000,1.186087e+09,135.500000
75%,8122.000000,477.000000,4.000000,1.435994e+09,300.500000
max,193609.000000,610.000000,5.000000,1.537799e+09,1404.000000


In [96]:
filt=final_df.query('totalratingcount>=140')
filt.head()

,movieId,title,userId,rating,timestamp,totalratingcount
0,1,Toy Story (1995),1,4.0,964982703,843.0
1,1,Toy Story (1995),5,4.0,847434962,843.0
2,1,Toy Story (1995),7,4.5,1106635946,843.0
3,1,Toy Story (1995),15,2.5,1510577970,843.0
4,1,Toy Story (1995),17,4.5,1305696483,843.0


In [43]:
train_df= filt.pivot_table(columns='userId', index='title', values='rating')
train_df.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5
10 Things I Hate About You (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,3.0,NaN,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (One Hundred and One Dalmatians) (1961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
train_df.fillna(0, inplace = True)
train_df.head()


userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,3.0,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (One Hundred and One Dalmatians) (1961),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
from scipy.sparse import csr_matrix
matrix=csr_matrix(train_df)
print(matrix)
print(type(matrix))

  (0, 14)	4.0
  (0, 17)	4.0
  (0, 21)	0.5
  (0, 40)	3.5
  (0, 60)	4.5
  (0, 66)	4.0
  (0, 68)	4.0
  (0, 71)	4.0
  (0, 87)	2.5
  (0, 90)	4.0
  (0, 103)	5.0
  (0, 123)	1.0
  (0, 139)	3.0
  (0, 141)	5.0
  (0, 146)	3.5
  (0, 151)	3.5
  (0, 157)	5.0
  (0, 174)	3.5
  (0, 210)	4.5
  (0, 229)	5.0
  (0, 246)	4.0
  (0, 277)	4.0
  (0, 295)	2.0
  (0, 314)	3.0
  (0, 315)	4.0
  :	:
  (637, 378)	2.0
  (637, 384)	3.0
  (637, 410)	0.5
  (637, 411)	4.0
  (637, 416)	4.0
  (637, 417)	4.0
  (637, 421)	3.0
  (637, 423)	2.0
  (637, 435)	2.5
  (637, 445)	3.0
  (637, 477)	4.5
  (637, 480)	5.0
  (637, 481)	4.5
  (637, 486)	1.5
  (637, 492)	5.0
  (637, 522)	4.0
  (637, 531)	4.0
  (637, 539)	1.5
  (637, 557)	4.0
  (637, 558)	3.0
  (637, 570)	5.0
  (637, 587)	2.5
  (637, 597)	4.5
  (637, 605)	3.0
  (637, 607)	4.0
<class 'scipy.sparse.csr.csr_matrix'>


In [98]:
from sklearn.neighbors import NearestNeighbors
knn_model=NearestNeighbors(metric="cosine", algorithm='brute')
knn_model.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [99]:
matrix.shape

(638, 608)

In [100]:
index=np.random.choice(matrix.shape[0])
distance , indices= knn_model.kneighbors(train_df.iloc[index,:].values.reshape(1,-1),n_neighbors=6)


In [101]:
print(distance)
print(indices)


[[0.         0.4634182  0.49458136 0.52253726 0.53695582 0.54541798]]
[[ 86  38 635 524 108 621]]


In [102]:
print(distance.flatten())


[0.         0.4634182  0.49458136 0.52253726 0.53695582 0.54541798]


In [103]:
print(f"The movies familiar with {train_df.index[index]}")
for i in range(len(distance.flatten())):
  print(train_df.index[indices.flatten()[i]],distance.flatten()[i])




The movies familiar with Blazing Saddles (1974)
Blazing Saddles (1974) 0.0
Animal House (1978) 0.4634182004544506
Young Frankenstein (1974) 0.494581363122061
Spaceballs (1987) 0.5225372623021074
Caddyshack (1980) 0.5369558157237815
Who Framed Roger Rabbit? (1988) 0.545417980791777
